# PSNR & SSIM

In [1]:
#import tensorflow as tf
import math
import numpy as np
from PIL import Image
import cv2
import os

In [2]:
def calculate_psnr(img1, img2):
    # img1 and img2 have range [0, 255]
    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    mse = np.mean((img1 - img2)**2)
    if mse == 0:
        return float('inf')
    return 20 * math.log10(255.0 / math.sqrt(mse))

def ssim(img1, img2):
    C1 = (0.01 * 255)**2
    C2 = (0.03 * 255)**2

    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    kernel = cv2.getGaussianKernel(11, 1.5)
    window = np.outer(kernel, kernel.transpose())

    mu1 = cv2.filter2D(img1, -1, window)[5:-5, 5:-5]  # valid
    mu2 = cv2.filter2D(img2, -1, window)[5:-5, 5:-5]
    mu1_sq = mu1**2
    mu2_sq = mu2**2
    mu1_mu2 = mu1 * mu2
    sigma1_sq = cv2.filter2D(img1**2, -1, window)[5:-5, 5:-5] - mu1_sq
    sigma2_sq = cv2.filter2D(img2**2, -1, window)[5:-5, 5:-5] - mu2_sq
    sigma12 = cv2.filter2D(img1 * img2, -1, window)[5:-5, 5:-5] - mu1_mu2

    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) *
                                                            (sigma1_sq + sigma2_sq + C2))
    return ssim_map.mean()


def calculate_ssim(img1, img2):
    '''calculate SSIM
    the same outputs as MATLAB's
    img1, img2: [0, 255]
    '''
    if not img1.shape == img2.shape:
        raise ValueError('Input images must have the same dimensions.')
    if img1.ndim == 2:
        return ssim(img1, img2)
    elif img1.ndim == 3:
        if img1.shape[2] == 3:
            ssims = []
            for i in range(3):
                ssims.append(ssim(img1, img2))
            return np.array(ssims).mean()
        elif img1.shape[2] == 1:
            return ssim(np.squeeze(img1), np.squeeze(img2))
    else:
        raise ValueError('Wrong input image dimensions.')
        
def img2np(img_path):
    img=Image.open(img_path)
    return np.array(img)

#gan切一半
def img2np_gan(img_path):
    img=Image.open(img_path)
    cropped = img.crop((512, 0, 1024, 512))
    return np.array(cropped)

#算兩個資料夾內相同圖片的psnr和ssim的平均
def mean_psnr_ssim(ori_path,deblur_path):
    total_psnr = 0
    total_ssim = 0
    for i in os.listdir(ori_path):
        ori_img_path = os.path.join(ori_path,i)
        deblur_img_path = os.path.join(deblur_path,i)
        total_psnr += calculate_psnr(img2np(ori_img_path),img2np_gan(deblur_img_path))  #gan切一半
        total_ssim += calculate_ssim(img2np(ori_img_path),img2np_gan(deblur_img_path))  #gan切一半
    print("平均psnr = ", total_psnr/len(os.listdir(ori_path)))
    print("平均ssim = ", total_ssim/len(os.listdir(ori_path)))

In [3]:
#原本gan
HP_ori_path = r"D:\psnr_ssim\原本pix2pix\HPtest2HP_origin_pix2pix"
HP_gan_path = r"D:\0719GAN資料\data_split_nonbi\HP\test"
SSA_ori_path = r"D:\psnr_ssim\原本pix2pix\SSAtest2SSA_origin_pix2pix"
SSA_gan_path = r"D:\0719GAN資料\data_split_nonbi\SSA\test"
TA_ori_path = r"D:\psnr_ssim\原本pix2pix\TAtest2TA_origin_pix2pix"
TA_gan_path = r"D:\0719GAN資料\data_split_nonbi\TA\test"

In [6]:
mean_psnr_ssim(HP_ori_path,HP_gan_path)
mean_psnr_ssim(SSA_ori_path,SSA_gan_path)
mean_psnr_ssim(TA_ori_path,TA_gan_path)

平均psnr =  24.367349633750887
平均ssim =  0.9192010104790864
平均psnr =  21.432286261085583
平均ssim =  0.8685824193784262
平均psnr =  22.83624236950441
平均ssim =  0.7994993156308975


In [7]:
#earlystop25 gan
HP_ori_path = r"D:\psnr_ssim\earlystop25\HPtestHP_earlystop_25epoch"
HP_gan_path = r"D:\0719GAN資料\data_split_nonbi\HP\test"
SSA_ori_path = r"D:\psnr_ssim\earlystop25\SSAtest2SSA_earlystop_25epoch"
SSA_gan_path = r"D:\0719GAN資料\data_split_nonbi\SSA\test"
TA_ori_path = r"D:\psnr_ssim\earlystop25\TAtest2TA_earlystop_25epoch"
TA_gan_path = r"D:\0719GAN資料\data_split_nonbi\TA\test"

In [8]:
mean_psnr_ssim(HP_ori_path,HP_gan_path)
mean_psnr_ssim(SSA_ori_path,SSA_gan_path)
mean_psnr_ssim(TA_ori_path,TA_gan_path)

平均psnr =  25.73136650935895
平均ssim =  0.9262808385076039
平均psnr =  21.738015261544597
平均ssim =  0.8971438048499826
平均psnr =  23.354728552339278
平均ssim =  0.8277227969218566
